# Detecting Building defects using CNN

### Importing Neccessary Libraries

In [1]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


### Image Data Agumentation

In [2]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [3]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(
    r'C:\Users\botla\Desktop\Rock_Classification_Dataset\Rock_Classification_Dataset\train_set',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(
    r'C:\Users\botla\Desktop\Rock_Classification_Dataset\Rock_Classification_Dataset\test_set',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='categorical') 

Found 700 images belonging to 5 classes.
Found 300 images belonging to 5 classes.


In [4]:
print(x_train.class_indices)#checking the number of classes

{'blue calcite': 0, 'limestone': 1, 'marble': 2, 'olivine': 3, 'red crystal': 4}


In [5]:
print(x_test.class_indices)#checking the number of classes

{'blue calcite': 0, 'limestone': 1, 'marble': 2, 'olivine': 3, 'red crystal': 4}


In [6]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 140, 1: 140, 2: 140, 3: 140, 4: 140})

### Creating the model

In [7]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=5, activation='softmax')) # softmax for more than 2



In [8]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6

### Compiling the model

In [9]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [10]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=50, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

C:\Users\botla\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
140/140 [==============================] - 44s 310ms/step - loss: 0.9498 - accuracy: 0.5986 - val_loss: 0.5348 - val_accuracy: 0.8200
Epoch 2/50
140/140 [==============================] - 4s 29ms/step - loss: 0.5380 - accuracy: 0.7557 - val_loss: 0.5483 - val_accuracy: 0.7800
Epoch 3/50
140/140 [==============================] - 4s 28ms/step - loss: 0.4946 - accuracy: 0.8014 - val_loss: 0.3791 - val_accuracy: 0.8800
Epoch 4/50
140/140 [==============================] - 4s 29ms/step - loss: 0.3831 - accuracy: 0.8371 - val_loss: 0.5210 - val_accuracy: 0.8167
Epoch 5/50
140/140 [==============================] - 4s 28ms/step - loss: 0.3763 - accuracy: 0.8429 - val_loss: 0.6413 - val_accuracy: 0.8067
Epoch 6/50
140/140 [==============================] - 4s 28ms/step - loss: 0.3547 - accuracy: 0.8614 - val_loss: 0.6080 - val_accuracy: 0.8500
Epoch 7/50
140/140 [==============================] - 4s 28ms/step - loss: 0.3249 - accuracy: 0.8700 - val_loss: 0.4677 - val_accuracy: 0.84

### Saving our model

In [15]:
# Save the model
classifier.save('rock.h5')

### Predicting our results

In [17]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("rock.h5") #loading the model for testing

In [25]:
img = image.load_img(r"C:\Users\botla\Desktop\Rock_Classification_Dataset\Rock_Classification_Dataset\train_set\marble\marble15.jpg",
                     grayscale=False,target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict_classes(x)#predicting the classes
pred

array([2], dtype=int64)

In [26]:
index=['blue calcite', 'limestone', 'marble', 'olivine', 'red crystal']
result=str(index[pred[0]])
result

'marble'